## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
keras = tf.keras

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
tf.test.is_built_with_cuda()

In [ ]:
def seq2seq_window_dataset(series, window_size, batch_size=32):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)
  

def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [ ]:
path = r'C:\Users\adria\1Python\Meu_novo_projeto\WIN_N_1K\WIN_N_1k.csv'
raw = pd.read_csv(path, index_col=0, parse_dates=True)

In [ ]:
df = pd.DataFrame()

In [ ]:
df['close'] = (raw['high'] + raw['low']) / 2

In [ ]:
media_movel = df['close'].rolling(window=2).mean()

In [ ]:
df['media_movel'] = media_movel

In [ ]:
df

In [ ]:
df['r'] = df['media_movel'].pct_change()

In [ ]:
df = df.dropna()

series = df['r'].values

time = np.arange(len(series))

In [ ]:
#################

#df = pd.DataFrame()
#df['close'] = (raw['high'] + raw['low']) / 2
#media_movel = df['close'].rolling(window=2).mean()
#df['media_movel'] = media_movel

#df['r'] = df['close'].pct_change()

#df = df.dropna()

#series = df['r'].values

#time = np.arange(len(series))

In [ ]:
split_time = 70000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

In [ ]:
def sequential_window_dataset(series, window_size):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=window_size, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(window_size + 1))
    ds = ds.map(lambda window: (window[:-1], window[1:]))
    return ds.batch(1).prefetch(1)

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

## WaveNet

In [ ]:
%%time
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = sequential_window_dataset(x_train, window_size)
valid_set = sequential_window_dataset(x_valid, window_size)

model = keras.models.Sequential([
  keras.layers.LSTM(100, return_sequences=True, stateful=True,
                         batch_input_shape=[1, None, 1]),
  keras.layers.LSTM(100, return_sequences=True, stateful=True),
  keras.layers.Dense(1),
])
optimizer = keras.optimizers.Adam()
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mse"])
reset_states = ResetStatesCallback()
model_checkpoint = keras.callbacks.ModelCheckpoint(
    "my_checkpoint.h5", save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=50)
model.fit(train_set, epochs=200,
          validation_data=valid_set,
          callbacks=[early_stopping, model_checkpoint, reset_states])

In [ ]:
model = keras.models.load_model("my_checkpoint.h5")

In [ ]:
cnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
cnn_forecast = cnn_forecast[split_time - window_size:-1, -1, 0]

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(time_valid, x_valid)
plot_series(time_valid, cnn_forecast)

In [ ]:
keras.metrics.mean_absolute_error(x_valid, cnn_forecast).numpy()

In [ ]:
keras.metrics.mean_squared_error(x_valid, cnn_forecast).numpy()

In [ ]:
df2 = pd.DataFrame()

df2['diff_moving_avg_plus_past'] = cnn_forecast
df2['x_valid'] = x_valid

ultimo_preco_fechamento = 113630.0

df2['preco_valid'] = (1 + df2['x_valid']).cumprod()
df2['preco_valid'] = ultimo_preco_fechamento * df2['preco_valid']

df2['desr_prev'] = (1 + df2['diff_moving_avg_plus_past']).cumprod()
df2['desr_prev'] = ultimo_preco_fechamento * df2['desr_prev']

In [ ]:
df2

In [ ]:
len(x_valid)

In [ ]:
len(x_train)

In [ ]:
df2['diff'] = df2['preco_valid']-df2['desr_prev']

In [ ]:
df2['diff'].mean()